In [ ]:
import os
import torch
import gc
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import RTDETR

In [1]:
def run_research_experiment(exp_name, yaml_file):
    print(f"\n--- Initializing RT-DETR Experiment: {exp_name} ---")
    
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(5) 
    
    model = RTDETR("rtdetr-l.pt")
    
    model.train(
        data=yaml_file,
        epochs=100,
        imgsz=640,        
        batch=-1,          
        project="floorplan_research_detr",
        name=exp_name,
        device=0,
        exist_ok=True,
        verbose=True
    )

    results_path = f"floorplan_research_detr/{exp_name}/results.csv"
    if os.path.exists(results_path):
        df = pd.read_csv(results_path)
        df.columns = [c.strip() for c in df.columns]
        
        fig, ax = plt.subplots(1, 2, figsize=(15, 5))
        ax[0].plot(df['epoch'], df['train/box_loss'], label='Train Loss')
        ax[0].plot(df['epoch'], df['val/box_loss'], label='Val Loss')
        ax[0].set_title(f'RT-DETR {exp_name} Loss')
        ax[0].legend()
        
        ax[1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50')
        ax[1].set_title(f'RT-DETR {exp_name} Accuracy')
        ax[1].legend()
        plt.show()

    best_path = f"floorplan_research_detr/{exp_name}/weights/best.pt"
    best_model = RTDETR(best_path)
    val_results = best_model.val(data=yaml_file, split='test', device=0, verbose=False)
    
    maps = val_results.box.maps 
    class_names = ["Door", "Wall", "Window"]
    
    print(f"\nFinal Analysis for {exp_name} (RT-DETR):")
    for i, name in enumerate(class_names):
        score = maps[i] if i < len(maps) else 0.0
        print(f"{name} Score: {score:.4f}")
    
    print(f"Overall Macro Score: {np.mean(maps):.4f}")
    
    del model
    del best_model
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Memory Cleared. Ready for next experiment.\n")
    print("-" * 40)



In [2]:
run_research_experiment("colorful_rtdetr", "colorful.yaml")


--- Initializing RT-DETR Experiment: colorful_rtdetr ---
Ultralytics 8.3.241  Python-3.10.19 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=colorful.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=colorful_rtdetr, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

C:\ProgramData\anaconda3\envs\ai_pipeline_env\lib\site-packages\torch\autograd\graph.py:744: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ..\aten\src\ATen\Context.cpp:87.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/100      2.11G      1.588      0.926     0.8091         40        640: 100% ━━━━━━━━━━━━ 218/218 3.4it/s 1:05<0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 14.3it/s 0.9s0.2s
                   all         26       1133       0.27      0.335      0.221      0.139

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      2/100      2.11G      1.056     0.6215     0.4103         31        640: 0% ──────────── 0/218  0.3s

C:\ProgramData\anaconda3\envs\ai_pipeline_env\lib\site-packages\torch\autograd\graph.py:744: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ..\aten\src\ATen\Context.cpp:87.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/100      2.11G        1.3     0.4476      0.531         21        640: 100% ━━━━━━━━━━━━ 218/218 3.6it/s 1:00<0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 14.4it/s 0.9s0.1s
                   all         26       1133      0.577      0.401      0.417      0.259

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      3/100      2.11G       1.54     0.3415     0.5945        108        640: 0% ──────────── 0/218  0.4s

...

C:\ProgramData\anaconda3\envs\ai_pipeline_env\lib\site-packages\torch\autograd\graph.py:744: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ..\aten\src\ATen\Context.cpp:87.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     99/100      2.13G     0.3459     0.3391     0.0754         13        640: 100% ━━━━━━━━━━━━ 218/218 3.6it/s 60.0s<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 14.3it/s 0.9s0.1s
                   all         26       1133      0.903      0.858      0.876      0.618

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
    100/100      2.13G      0.361     0.3754      0.086         40        640: 0% ──────────── 0/218  0.3s

C:\ProgramData\anaconda3\envs\ai_pipeline_env\lib\site-packages\torch\autograd\graph.py:744: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ..\aten\src\ATen\Context.cpp:87.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    100/100      2.13G     0.3418      0.344    0.07228         26        640: 100% ━━━━━━━━━━━━ 218/218 3.6it/s 1:01<0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 14.1it/s 0.9s0.1s
                   all         26       1133      0.897      0.858      0.872      0.616

100 epochs completed in 1.802 hours.
Optimizer stripped from C:\Users\Artem\Jupyter notebooks\DL_project\floorplan_research_detr\colorful_rtdetr\weights\last.pt, 66.2MB
Optimizer stripped from C:\Users\Artem\Jupyter notebooks\DL_project\floorplan_research_detr\colorful_rtdetr\weights\best.pt, 66.2MB

Validating C:\Users\Artem\Jupyter notebooks\DL_project\floorplan_research_detr\colorful_rtdetr\weights\best.pt...
Ultralytics 8.3.241  Python-3.10.19 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
rt-detr-l summary: 310 layers, 31,989,905 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P    

KeyError: 'train/box_loss'

In [ ]:
run_research_experiment("bw_rtdetr", "bw.yaml")